# Libraries

In [1]:
import pandas as pd
import pyodbc
import os
from datetime import datetime
pd.io.formats.excel.ExcelFormatter.header_style = None

# Parameters

In [2]:
today = datetime.today()
year = today.year
week = today.isocalendar()[1]

# Connections

In [3]:
# Connection to DWH
## Server:SFEDWH01|DB:Gnm_DWH
conn1 = pyodbc.connect('Driver={SQL Server};'
                       'Server=SFEDWH01;'
                       'Database=Gnm_DWH;'
                       'Trusted_Connection=yes;')

## Server:SFEDWH01|DB:Gnm_MasterOp
conn2 = pyodbc.connect('Driver={SQL Server};'
                       'Server=SFEDWH01;'
                       'Database=Gnm_MasterOp;'
                       'Trusted_Connection=yes;')

# Queries

In [4]:
# Query time data
qtiempo = '''SELECT TmpFecha FROM Gnm_DWH.dbo.Dim_Tiempo WHERE TmpAnioSemanaGenomma={0} AND TmpSemanaAnioGenomma={1} AND TmpDiaSemana=6'''
# Import time data
dtiempo = pd.read_sql(qtiempo.format(year, week), conn1)
# Date from last saturday
date = dtiempo['TmpFecha'].loc[0].to_pydatetime()

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Import Data

In [5]:
path = r'\\NASPRO.infovisiontv.com\DWH01\Cognos\Genomma lab USA\US Data\Amazon\{0}\W {1} {0}'
path = path.format(year, str(week).zfill(2))
files = [file for file in os.listdir(path) if 'Weekly' in file]

In [6]:
def colnum_string(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

In [7]:
files = os.listdir(path)[:2]
cols_3p = 'A, F:I, {}'.format(colnum_string(61 + week))
cols_1p = 'A, F:I, {}'.format(colnum_string(174 + week))

In [8]:
%%time
df = pd.DataFrame()
cols = ['ASIN', 'Product Title', 'UPC', 'List Price to Calculate Units', 'Brand'] + [date]
for file in files:
    try:
        aux = pd.read_excel(path + '\\' +  file, header=1, sheet_name='Reporte Julio Velazque', usecols=cols_3p)
    except:
        aux = pd.read_excel(path + '\\' +  file, header=1, usecols=cols_1p)
    df = pd.concat([df, aux[cols]])

Wall time: 6.78 s


# Data Cleaning

In [9]:
df.isnull().sum()

ASIN                             25
Product Title                    26
UPC                              26
List Price to Calculate Units    26
Brand                            24
2022-03-12 00:00:00              24
dtype: int64

In [10]:
df = df[df['Product Title'].notnull()]

df.reset_index(drop=True, inplace=True)

In [11]:
df['UPC'] = df['UPC'].map(int)

eans = df.loc[df['List Price to Calculate Units'].isnull(), 'UPC'].values

for ean in eans:
    price = df['List Price to Calculate Units'][(df['List Price to Calculate Units'].notnull())&(df['UPC']==ean)].values[0]
    df.loc[df['UPC']==ean, 'List Price to Calculate Units'] = price

In [12]:
df.isnull().sum()

ASIN                             0
Product Title                    0
UPC                              0
List Price to Calculate Units    0
Brand                            0
2022-03-12 00:00:00              0
dtype: int64

In [13]:
df['Product Title'] = df['Product Title'].map(lambda title: title[:80] if len(title) > 80 else title)

df['Sales Rank'] = range(1, df.shape[0] + 1)

df = df[['ASIN', 'Sales Rank', 'Product Title', 'UPC', 'List Price to Calculate Units', 'Brand'] + [date]].copy()

In [14]:
header = pd.DataFrame(df.columns).T
header.columns = df.columns

df = pd.concat([header, df])
df.columns = [1 for i in range(len(df.columns))]

In [15]:
df.to_excel(path + '\\AmazonSellOut.xlsx', sheet_name='Hoja1', index=False)